<a href="https://colab.research.google.com/github/claudioquevedo/Pos-Graduacao-em-IA/blob/master/Sistemas_Especialistas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Avaliação de Conhecimentos Fotográficos**

In [55]:
# @title Instalação e importação de frameworks
#!pip install tensorflow

# Realiza as importações dos frameworks necessários
import IPython
import numpy as np
import tensorflow as tf
import os
import warnings
import logging
import sys

from IPython.display import display
from IPython.display import HTML
from enum import Enum
from keras.preprocessing import sequence

warnings.filterwarnings('ignore')

In [3]:
# @title Classe de exibição de textos formatados
# Classe para escrever texto formatado
class MessageType(Enum):
  mtTitulo    = "font-size:18px; font-weight:bold; color:#C6E5FF; text-decoration: underline; line-height: 3;"
  mtCabecalho = "font-size:16px; font-weight:bold; color:#C6E5FF; text-decoration: underline; line-height: 2;"
  mtPergunta  = "color: #AAD3FF; line-height: 1.5;"
  mtErro      = "color:#EB564B; "
  mtNormal    = ""

class Display:
  @staticmethod
  def ShowMessage(Message: str, MessageType: MessageType):
    if MessageType == MessageType.mtNormal:
      print(Message)
    else:
      display(HTML(f'<span style="{MessageType.value}">{Message}</span>'))

In [4]:
# @title Classes Data Transfer
# Classe de tipo de questionário
class Questionario:
  def __init__(self, titulo: str, introducao: str):
    self.Titulo = titulo
    self.Introducao = introducao

  def __repr__(self):
    return f"Questionario(Titulo={self.Titulo}, Introducao={self.Introducao})"
#------------------------------------------------------------------------

# Classe que conterá as respostas dadas pelo usuário
class RespostaParaAnalise:
  def __init__(self, categoria: int, CaptionCategoria: str, pesoCategoria: float, maximoPontos: float, ponto: float):
    self.Categoria = categoria
    self.CaptionCategoria = CaptionCategoria
    self.PesoCategoria = pesoCategoria
    self.MaximoPontos = maximoPontos
    self.Ponto = ponto

  def __repr__(self):
    return f"RespostaParaAnalise(Categoria={self.Categoria}, CaptionCategoria={self.CaptionCategoria}, PesoCategoria={self.PesoCategoria}, MaximoPontos={self.MaximoPontos}, Ponto={self.Ponto})"
#------------------------------------------------------------------------

# Classe que conterá o resumo da pontuação do usuário, agregado por categoria
class CategoriaParaAnalise:
  def __init__(self, categoria: int,  captionCategoria: str, pesoCategoria: float, maximoPontos: float, pontosObtidos: float, percentual: float):
    self.Categoria = categoria
    self.CaptionCategoria = captionCategoria
    self.PesoCategoria = pesoCategoria
    self.MaximoPontos = maximoPontos
    self.PontosObtidos = pontosObtidos
    self.Percentual = percentual

  @property
  def PercentualAcerto(self):
    if self.MaximoPontos == 0:
      return self.Percentual
    else:
      return self.PontosObtidos * 100 / self.MaximoPontos

  def __repr__(self):
    return f"CategoriaParaAnalise(CaptionCategoria={self.CaptionCategoria}, categoria={self.Categoria}, pesoCategoria={self.PesoCategoria}, máximo de pontos={self.MaximoPontos}, pontos obtidos: {self.PontosObtidos})"
#------------------------------------------------------------------------

class AnaliseRespostaCategoria:
  def __init__(self,indice: int, categoria: int, minimo: float, maximo: float, resposta: str):
    self.Indice = indice
    self.Categoria = categoria
    self.Minimo = minimo
    self.Maximo = maximo
    self.Resposta = resposta

  def __repr__(self):
    return f"AnaliseRespostaCategoria(Categoria={self.Categoria}, Minimo={self.Minimo}, maximo={self.Maximo}, Resposta={self.Resposta})"
#------------------------------------------------------------------------


class RespostaItem:
  def __init__(self, index: int, caption: str, grade: float):
    self.Index = index
    self.Caption = caption
    self.Grade = grade

  def __repr__(self):
    return f"RespostaItem(Index={self.Index}, Caption={self.Caption}, Grade={self.Grade})"
#------------------------------------------------------------------------

class PerguntaItem:
  def __init__(self, index: int, caption: str, respostas: list[RespostaItem]):
    self.Index = index
    self.Caption = caption
    self.Respostas = respostas

  def __repr__(self):
    return f"PerguntaItem(Index={self.Index}, Caption={self.Caption}, Respostas={self.Respostas})"
#------------------------------------------------------------------------

class CategoriaPergunteItem:
  def __init__(self, index: int, caption: str, peso: float, perguntas: list[PerguntaItem]):
    self.Index = index
    self.Caption = caption
    self.Peso = peso
    self.Perguntas = perguntas

  def __repr__(self):
    return f"CategoriaPergunteItem(Index={self.Index}, Caption={self.Caption}, Peso={self.Peso}, Perguntas={self.Perguntas})"
#------------------------------------------------------------------------

In [5]:
# @title Monta questionário e avaliação final
#Builder para criar o título e a descrição do questionário
class QuestionarioBuilder:
  @staticmethod
  def Build(self):
    self.DescicaoQuestionario = Questionario("Avaliação de Conhecimentos Básicos de Fotografia",
                                             "Com o intuito de avaliar suas capacidades fotográficas e o que conseguiu apreender durante\n" +
                                             "o curso básico de fotografia, além de guiá-lo no desenvolvimento de suas capacidades e ajudá-lo\n" +
                                             "a identificar seus pontos fortes e fracos, apresentamos a avaliação abaixo.\n\n" +
                                             "Leia atentamente as perguntas e as respostas e indique a que melhor responde às questões propostas.\n" +
                                             "Em tempo, não pense que todas as questões são feitas de ""certo/errado"", pois em alguns casos podem \n" +
                                             "existir várias respostas certas, de acordo com o seu nível de conhecimento. Nesse caso, escolha a que\n" +
                                             "achar a mais completa ou mais correta.\n\n" +
                                             "Vamos começar.")

    return self.DescicaoQuestionario
#------------------------------------------------------------------------

# Builder para criar a estrutura de perguntas
class PerguntasBuilder:
  @staticmethod
  def Build():
    listaPeguntas = [
        CategoriaPergunteItem(1, "Conceitos Básicos", 3,
                              [PerguntaItem(1, "Sobre o foco em fotografias de pessoas, qual das opções você considera a mais pertinente?",
                                            [RespostaItem(1, "Deve sempre abranger todo o rosto da pessoa", 2),
                                             RespostaItem(2, "Deve estar sempre cravado nos olhos da pessoa", 3),
                                             RespostaItem(3, "Deve estar sempre na região mais próxima da câmera, evidenciando a profundidade de campo", 1),
                                             RespostaItem(4, "Pode estar em qualquer lugar da foto", 0)]),
                              PerguntaItem(2, "Sobre profundidade de campo, qual das opções é a correta?",
                                            [RespostaItem(1, "Quanto mais aberto o diafragma, maior a profundidade de campo", 0),
                                             RespostaItem(2, "Quanto maior a velocidade do obturador, maior a profundidade de campo", 0),
                                             RespostaItem(3, "Quanto mais fechado o diafragma, maior a profundidade de campo", 3),
                                             RespostaItem(4, "Quanto maior o ISO, maior a profundidade de campo", 0)]),
                              PerguntaItem(3, "Sobre ISO, qual das opções é a correta?",
                                            [RespostaItem(1, "Determina a sensibilidade do sensor à luz que chega até ele", 3),
                                             RespostaItem(2, "Determina a quantidade de luz que chega ao sensor", 0),
                                             RespostaItem(3, "Determina a velocidade com que o obturador abre e fecha", 0),
                                             RespostaItem(4, "Determina se a fotografia ficará ou não no foco", 0)]),
                               PerguntaItem(7, "Sobre os efeitos da velocidade do obturador no resultado da foto, pode-se afirmar que:",
                                            [RespostaItem(1, "Permite que o sensor ""perceba"" mais ou menos luz", 0),
                                             RespostaItem(2, "Aumenta a profundidade de campo da cena", 0),
                                             RespostaItem(3, "Possui como unidade de medida a letra ""f"" e quanto maior o valor, maior a quantidade de luz que chega ao sensor", 0),
                                             RespostaItem(4, "É representada em segundos e suas frações e tem a capacidade de ""congelar"" a cena quando alta o suficiente", 3)]),
                               PerguntaItem(8, "Sobre a abertura do diafragma, qual das opções é a mais completa?",
                                            [RespostaItem(1, "Permite que o sensor ""perceba"" mais ou menos luz", 1),
                                             RespostaItem(2, "Pode interferir no foco da foto, uma vez que que influencia a profundidade de campo", 2),
                                             RespostaItem(3, "Além da opção anterior, possui como unidade de medida a letra ""f"" e quanto menor seu valor, mais luz chega ao sensor", 3),
                                             RespostaItem(4, "Está intimamente ligada à câmera e ao tamanho do sensor", 0)])
                              ]),
        CategoriaPergunteItem(2, "Composição", 2,
                              [PerguntaItem(4, "Sobre a regra dos terços, qual das opções você considera a mais pertinente?",
                                            [RespostaItem(1, "É a melhor forma de posicionar os componentes da foto no quadro", 1),
                                             RespostaItem(2, "Não deve ser usada, de forma alguma, por fotógrafos com conhecimentos avançados ou profissionais", 0),
                                             RespostaItem(3, "É mais usada em fotografias de paisagem", 0),
                                             RespostaItem(4, "É uma entre muitas formas que o fotógrafo tem para compor a cena", 3)]),
                               PerguntaItem(5, "Sobre a técnica de composição utilizando ""linhas guia"", escolha a melhor opção:",
                                            [RespostaItem(1, "São linhas na cena onde os pontos de atenção devem estar posicionados", 1),
                                             RespostaItem(2, "São linhas imaginárias na cena que dão simetria à composição", 2),
                                             RespostaItem(3, "São representadas pelos contornos dos objetos da cena", 0),
                                             RespostaItem(4, "São melhor utilizadas quando sugerem inconscientemente ao observador o caminho que leva ao ponto de interesse da foto", 3)]),
                               PerguntaItem(6, "Indique a melhor descrição para o uso da técnica da simetria:",
                                            [RespostaItem(1, "O assunto da foto deve sempre estar centralizado e o formato da foto deve ser quadrado", 1),
                                             RespostaItem(2, "Pelo menos dois lados da foto devem ser simétricos, não importando a orientação e nem o formato da foto", 3),
                                             RespostaItem(3, "Os resultados de fotografias simétricas não são muito agradáveis aos olhos do observador", 0),
                                             RespostaItem(4, "Nunca pode ser utilizado em fotos de retrato porque o rosto humano nunca é simétrico", 0)])
                              ])
    ]
    return listaPeguntas
#------------------------------------------------------------------------

# Builder para criar a estrutura de análise de respostas
class ResultadosAnaliseBuilder:
  @staticmethod
  def Build():
    listaResultadoAnaliseCategoria = [AnaliseRespostaCategoria(1, 1, 0, 30,   "Você ainda precisa estudar muito. Todas as suas boas fotos são, de uma forma geral, \n" +
                                                                                   "obra da sorte."),
                                           AnaliseRespostaCategoria(2, 1, 31, 50,  "Você já tem um conhecimento básico e pode tirar boas fotos, mas ainda conta muito \n" +
                                                                                   "com o fator ""sorte"" para conseguir fotos com alguma qualidade."),
                                           AnaliseRespostaCategoria(3, 1, 51, 80,  "Você possui um bom conhecimento dos conceitos básicos de fotografia, o que lhe permite \n" +
                                                                                   "um bom aproveitamento das fotos tiradas."),
                                           AnaliseRespostaCategoria(4, 1, 81, 100, "Você possui todos os conhecimentos básicos necessários para garantir uma boa fotografia, \n" +
                                                                                   "embora somente esses conhecimentos não garantam uma foto sem as técnicas de composição \n" +
                                                                                   "adequadas."),
                                           AnaliseRespostaCategoria(5, 2, 0, 30,   "Você não consegue, de forma consciente, obter boas fotografias. Ainda depende muito da sorte."),
                                           AnaliseRespostaCategoria(6, 2, 31, 50,  "Você consegue fazer boas fotos, mas mais estudo pode garantir a sua independência da sorte."),
                                           AnaliseRespostaCategoria(7, 2, 51, 80,  "Você já consegue ótimas fotos e de forma consciente. Com esse nível de conhecimento,o fator \n" +
                                                                                   "sorte já está quase eliminado."),
                                           AnaliseRespostaCategoria(8, 2, 81, 100, "Você já é capaz de fazer excelentes fotografias e, embora a sorte seja uma grande aliada \n" +
                                                                                   "do fotógrafo e não deva ser desprezada, suas fotos já têm o potencial de impactar somente \n" +
                                                                                   "pelos seus conhecimentos.")]

    return listaResultadoAnaliseCategoria

In [6]:
# @title Estrutura de classes para criar base de conhecimento
## a estrutura de resultados das análises das categorias deverá ser um array de array

class KnowledgeBase:

  def __init__ (self):
    self.__respostas = []
    self.__resultados = []
    self.Entradas = []
    self.Saidas = []

  def Clear(self):
    self.__respostas.clear()
    self.__resultados.clear()
    return self

  def Add (self, entradas, saidas):
    self.__respostas.append(entradas)
    self.__resultados.append(saidas)
    return self

  def Finalize (self):
    # Preencher as sublistas com "0"
    max_length = max(map(len, self.__respostas), default=0)
    vetor_preenchido = [sublista + [0] * (max_length - len(sublista)) for sublista in self.__respostas]

    vetor_normalizado = sequence.pad_sequences(self.__respostas, value=0, padding="post", maxlen=len(self.__respostas[0]))

    self.Entradas = (vetor_preenchido - np.min(vetor_normalizado)) / (np.max(vetor_normalizado) - np.min(vetor_normalizado))

    #self.Entradas = np.array(self.__respostas)
    self.Saidas = np.array(self.__resultados)
    return self

In [54]:
# @title Cria e configura a rede neural para análise das respostas e faz seu treinamento
class RedeNeural:

  def Compile (self, respostasTreino, resultadoTreino, epochNumber = 1500, loss_method = 'binary_crossentropy', SaveData = True, UseSavedData = True, ForceTraining = True):
    __learning_rate = 0.001
    __l2_regularization = 0.001
    __filePath = "Questionario_Forografia.h5"
    __lerRede = UseSavedData and os.path.exists(__filePath)

    original_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')

    if __lerRede:
      self.model = tf.keras.models.load_model(__filePath)
    else:
      # Construindo o modelo
      self.model = tf.keras.Sequential([
          tf.keras.layers.Dense(len(respostasTreino[0]), activation='relu', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(__l2_regularization)),
          #tf.keras.layers.Conv2D(32, kernel_size=(1, 2), activation='relu', input_shape=(-1, 4, 1)),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(128, activation='relu', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(__l2_regularization)),
          #tf.keras.layers.Dense(15, activation='relu', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(__l2_regularization)),
          #tf.keras.layers.Dense(8, activation='relu', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(__l2_regularization)),
          #tf.keras.layers.Dense(4, activation='relu', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(__l2_regularization)),
          tf.keras.layers.Dense(len(resultadoTreino[0]), activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(__l2_regularization))
      ])

      # Compilando o modelo
      self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=__learning_rate), loss=loss_method, metrics=["accuracy"])

    # Treinando o modelo
    if ForceTraining:
      self.model.fit(respostasTreino, resultadoTreino, epochs = epochNumber)

    # Avaliar o modelo
    self.loss, self.accuracy = self.model.evaluate(respostasTreino, resultadoTreino)

    if SaveData:
      self.model.save(__filePath)

    sys.stdout = original_stdout

    return self

In [8]:
# @title Cria matrizes de treinamento da rede neural e executa o treinamento
class KnowledgeBaseBuilder:
  @staticmethod
  def Build(categoriasList: list, epochs = 1500, SaveData = True, UseSavedData = True, ForceTraining = True, loss_method = 'binary_crossentropy'):
    knowledgeBase = KnowledgeBase()
    redeNeural = RedeNeural()

    # Adiciona os dados de treinamento
    knowledgeBase.Clear() \
                 .Add([1, 3, 3, 3, 3, 3], [1]) \
                 .Add([1, 3, 3, 3, 3, 2], [0.93]) \
                 .Add([1, 3, 3, 3, 3, 1], [0.87]) \
                 .Add([1, 3, 3, 3, 3, 0], [0.8]) \
                 .Add([1, 3, 0, 3, 3, 3], [0.8]) \
                 .Add([1, 3, 0, 3, 3, 2], [0.73]) \
                 .Add([1, 3, 0, 3, 3, 1], [0.67]) \
                 .Add([1, 3, 0, 3, 3, 0], [0.6]) \
                 .Add([1, 3, 3, 0, 3, 3], [0.8]) \
                 .Add([1, 3, 3, 0, 3, 2], [0.73]) \
                 .Add([1, 3, 3, 0, 3, 1], [0.67]) \
                 .Add([1, 3, 3, 0, 3, 0], [0.6]) \
                 .Add([1, 3, 3, 3, 0, 3], [0.8]) \
                 .Add([1, 3, 3, 3, 0, 2], [0.73]) \
                 .Add([1, 3, 3, 3, 0, 1], [0.67]) \
                 .Add([1, 3, 3, 3, 0, 0], [0.6]) \
                 .Add([1, 3, 0, 0, 3, 3], [0.6]) \
                 .Add([1, 3, 0, 0, 3, 2], [0.53]) \
                 .Add([1, 3, 0, 0, 3, 1], [0.47]) \
                 .Add([1, 3, 0, 0, 3, 0], [0.4]) \
                 .Add([1, 3, 3, 0, 0, 3], [0.6]) \
                 .Add([1, 3, 3, 0, 0, 2], [0.53]) \
                 .Add([1, 3, 3, 0, 0, 1], [0.47]) \
                 .Add([1, 3, 3, 0, 0, 0], [0.4]) \
                 .Add([1, 3, 0, 3, 0, 3], [0.6]) \
                 .Add([1, 3, 0, 3, 0, 2], [0.53]) \
                 .Add([1, 3, 0, 3, 0, 1], [0.47]) \
                 .Add([1, 3, 0, 3, 0, 0], [0.4]) \
                 .Add([1, 2, 3, 3, 3, 3], [0.93]) \
                 .Add([1, 2, 3, 3, 3, 2], [0.87]) \
                 .Add([1, 2, 3, 3, 3, 0], [0.73]) \
                 .Add([1, 2, 0, 3, 3, 3], [0.73]) \
                 .Add([1, 2, 0, 3, 3, 2], [0.67]) \
                 .Add([1, 2, 0, 3, 3, 1], [0.6]) \
                 .Add([1, 2, 0, 3, 3, 0], [0.53]) \
                 .Add([1, 2, 3, 0, 3, 3], [0.73]) \
                 .Add([1, 0, 3, 0, 0, 0], [0.2]) \
                 .Add([1, 0, 0, 3, 0, 3], [0.4]) \
                 .Add([1, 0, 0, 3, 0, 2], [0.33]) \
                 .Add([1, 0, 0, 3, 0, 1], [0.27]) \
                 .Add([1, 0, 0, 3, 0, 0], [0.2]) \
                 .Add([2, 3, 3, 3, 0, 0], [1]) \
                 .Add([2, 3, 2, 3, 0, 0], [0.89]) \
                 .Add([2, 3, 1, 3, 0, 0], [0.78]) \
                 .Add([2, 3, 0, 3, 0, 0], [0.67]) \
                 .Add([2, 3, 3, 1, 0, 0], [0.78]) \
                 .Add([2, 3, 2, 1, 0, 0], [0.67]) \
                 .Add([2, 3, 1, 1, 0, 0], [0.56]) \
                 .Add([2, 3, 0, 1, 0, 0], [0.44]) \
                 .Add([2, 3, 3, 0, 0, 0], [0.67]) \
                 .Add([2, 3, 2, 0, 0, 0], [0.56]) \
                 .Add([2, 3, 1, 0, 0, 0], [0.44]) \
                 .Add([2, 3, 0, 0, 0, 0], [0.33]) \
                 .Add([2, 2, 3, 3, 0, 0], [0.89]) \
                 .Add([2, 2, 2, 3, 0, 0], [0.78]) \
                 .Add([2, 2, 1, 3, 0, 0], [0.67]) \
                 .Add([2, 2, 0, 3, 0, 0], [0.56]) \
                 .Add([2, 2, 3, 1, 0, 0], [0.67]) \
                 .Add([2, 2, 2, 1, 0, 0], [0.56]) \
                 .Add([2, 2, 1, 1, 0, 0], [0.44]) \
                 .Add([2, 0, 0, 0, 0, 0], [0]) \
                 .Add([2, 0, 3, 3, 0, 0], [0.67]) \
                 .Finalize()

    # Compila dos dados de treinamento
    return redeNeural.Compile(knowledgeBase.Entradas, knowledgeBase.Saidas, epochs, loss_method, SaveData, UseSavedData, ForceTraining)

In [9]:
# @title Encapsula a classe que utiliza a rede neural numa estrutura que controla a existência ou não de um dataset já criado
class RedeNeuralBuilder:
  def __init__(self, listaPerguntas):
    self.__listaPerguntas = listaPerguntas
    self.__saveData = True
    self.__useSavedData = True
    self.__forceTraining = True
    self.__epochs = 500
    self.__lossMethod = 'binary_crossentropy'

  def SaveData(self):
    self.__saveData = True
    return self

  def DontSaveData(self):
    self.__saveData = False
    return self

  def UseSavedData(self):
    self.__useSavedData = True
    return self

  def DontUseSavedData(self):
    self.__useSavedData = False
    return self

  def ForceTrainingModel(self):
    self.__forceTraining = True
    return self

  def DontTrainModel(self):
    self.__forceTraining = False
    return self

  def UseEpochs(self, epochs):
    self.__epochs = epochs
    return self

  def WithLossMethod(self, loss_method):
    self.__lossMethod = loss_method
    return self

  def Create(self):
    if len(self.__listaPerguntas) == 0:
      self.__listaPerguntas = PerguntasBuilder.Build()

    knowledgeBaseModel = KnowledgeBaseBuilder.Build(self.__listaPerguntas, self.__epochs, self.__saveData, self.__useSavedData, self.__forceTraining, self.__lossMethod)
    self.loss = knowledgeBaseModel.loss
    self.accuracy = knowledgeBaseModel.accuracy

    return knowledgeBaseModel.model

In [10]:
# @title Classe para ler opções de resposta
class Keyboard:
  @staticmethod
  def input(opcoes_disponiveis):
    retorno = input("Digite sua resposta (ou 'S' para sair): ").upper()

    if (retorno != "S") and (retorno not in opcoes_disponiveis):
      Display.ShowMessage("", MessageType.mtNormal)
      Display.ShowMessage(f"A sua resposta deve estar entre as opções {opcoes_disponiveis} ou ""S"" para sair", MessageType.mtErro)
      Display.ShowMessage("", MessageType.mtNormal)

      retorno = Keyboard.input(opcoes_disponiveis)

    return retorno

In [11]:
# @title Classe para análise das respostas, no formato procedural, dadas em cada categoria
class AnaliseRespostasProcedural:
  @staticmethod
  def AnalisarRespostas(Respostas: list[RespostaParaAnalise]):
    listaCategoriaParaAnalises = []

    if len(Respostas) > 0:
      categoriaControle = 0

      for resposta in Respostas:
        if resposta.Categoria != categoriaControle:
          categoriaControle = resposta.Categoria
          listaCategoriaParaAnalises.append(CategoriaParaAnalise(resposta.Categoria,
                                                                 resposta.CaptionCategoria,
                                                                 resposta.PesoCategoria,
                                                                 sum(resposta.MaximoPontos for resposta in Respostas if resposta.Categoria == categoriaControle),
                                                                 sum(resposta.Ponto for resposta in Respostas if resposta.Categoria == categoriaControle),
                                                                 0))

    return listaCategoriaParaAnalises

In [58]:
# @title Classe para análise das respostas, usando a rede neural, dadas em cada categoria
class AnaliseRespostasIA:
  @staticmethod
  def AnalisarRespostas(Respostas: list[RespostaParaAnalise]):

    #-----------------------------------------------------------
    def SortCategoria(respostaParaAnalise: RespostaParaAnalise):
      return respostaParaAnalise.Categoria
    #-----------------------------------------------------------


    original_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')

    listaCategoriaParaAnalises = []

    if len(Respostas) > 0:
      Respostas.sort(key=SortCategoria)
      knowledgeBase = KnowledgeBase()

      categoriaControle = Respostas[0].Categoria

      respostas = [categoriaControle]

      for resposta in Respostas:
        if resposta.Categoria == categoriaControle:
          respostas.append(resposta.Ponto)
        else:
          knowledgeBase.Add(respostas, [])
          categoriaControle = resposta.Categoria
          respostas = [categoriaControle]
          respostas.append(resposta.Ponto)

      knowledgeBase.Add(respostas, []) \
                   .Finalize()

      redeNeuralBuilder = RedeNeuralBuilder([])
      predictions = redeNeuralBuilder.DontSaveData() \
                                     .DontTrainModel() \
                                     .UseSavedData() \
                                     .Create() \
                                     .predict(knowledgeBase.Entradas)

      listaPerguntas = PerguntasBuilder.Build()

      for i in range(len(predictions)):
        resposta = round(float(predictions[i]), 2)*100
        categoria = next((categoria for categoria in listaPerguntas if categoria.Index == i + 1))
        listaCategoriaParaAnalises.append(CategoriaParaAnalise(i + 1,
                                                               categoria.Caption,
                                                               categoria.Peso,
                                                               0,
                                                               0,
                                                               predictions[i][0]))

    sys.stdout = original_stdout

In [28]:
# @title Classe para exibir os resultados da análise final
class ResultadosAnalise:

  @staticmethod
  def Exibir(self, listaCategoriaParaAnalises, tipoAnalise: str):
    Display.ShowMessage("", MessageType.mtNormal)

    if len(listaCategoriaParaAnalises) > 0:
      Display.ShowMessage(f"Análise das Respostas Dadas ({tipoAnalise})", MessageType.mtCabecalho)


      listaRespostas = ResultadosAnaliseBuilder.Build();

      #TO DO: Criar uma classe para respostas de considerações finais, baseadas no peso e pontuação em cada categoria

      for categoriaParaAnalise in listaCategoriaParaAnalises:
        Display.ShowMessage(categoriaParaAnalise.CaptionCategoria, MessageType.mtPergunta)
        resposta = next((itemResposta.Resposta for itemResposta in listaRespostas if (itemResposta.Minimo <= categoriaParaAnalise.PercentualAcerto <= itemResposta.Maximo) and itemResposta.Categoria == categoriaParaAnalise.Categoria), "<não identificado>")
        Display.ShowMessage(resposta, MessageType.mtNormal)
        Display.ShowMessage("", MessageType.mtNormal)

      Display.ShowMessage("", MessageType.mtNormal)
    else:
      Display.ShowMessage("Nenhuma resposta foi fornecida para análise!", MessageType.mtErro)

In [29]:
# @title Classe que exibe o questionário e recolhe as respostas a cada pergunta

class AvaliacaoFotografo:
  def __init__(self):
    self.finalizarExecucao = False;
    self.respostasAnalise = []
    self.listaPerguntas = PerguntasBuilder.Build()
    self.Introducao = QuestionarioBuilder.Build(self)

  #-----------------------------------------------------------------------------
  def Le_Resposta(self, opcoes_disponiveis):
    if self.finalizarExecucao:
      return

    retorno = Keyboard.input(opcoes_disponiveis)

    if retorno == "S":
      self.finalizarExecucao = True

    return retorno
  #-----------------------------------------------------------------------------

  def IniciarAvaliacao(self):
    #Introdução
    Display.ShowMessage(self.Introducao.Titulo, MessageType.mtTitulo)
    Display.ShowMessage("Introdução", MessageType.mtCabecalho)
    Display.ShowMessage(self.Introducao.Introducao, MessageType.mtNormal)
    Display.ShowMessage("", MessageType.mtNormal)

    # Mostra a categoria da pergunta
    for categoria in self.listaPerguntas:
      Display.ShowMessage(categoria.Caption, MessageType.mtCabecalho)

      # Mostra a pergunta da vez
      for pergunta in categoria.Perguntas:
        Display.ShowMessage(pergunta.Caption, MessageType.mtPergunta)

        # Mostra as opções de resposta à pergunta da vez
        for resposta in pergunta.Respostas:
          Display.ShowMessage(f"\t{resposta.Index} - {resposta.Caption}", MessageType.mtNormal)

        # Espera que o usuário responda
        resposta_dada = self.Le_Resposta(["1", "2", "3", "4"])

        Display.ShowMessage(chr(175) * 45, MessageType.mtPergunta)

        # Sai da aplicação caso o usuário decida
        if self.finalizarExecucao:
          break
        else:
          Display.ShowMessage("", MessageType.mtNormal)

        # Valida e pega o peso da resposta dada
        for resposta in pergunta.Respostas:
           if int(resposta_dada) == resposta.Index:
              self.respostasAnalise.append(RespostaParaAnalise(categoria.Index, categoria.Caption, categoria.Peso, max(resposta.Grade for resposta in pergunta.Respostas), resposta.Grade))
              break

      if self.finalizarExecucao:
        break
      else:
        Display.ShowMessage("", MessageType.mtNormal)

    if self.finalizarExecucao:
      Display.ShowMessage("", MessageType.mtNormal)
      Display.ShowMessage("O usuário solicitou a finalização da avaliação!", MessageType.mtErro)

    resultadosAnalise = AnaliseRespostasProcedural.AnalisarRespostas(self.respostasAnalise)
    ResultadosAnalise.Exibir(self, resultadosAnalise, "análise procedural")
    Display.ShowMessage("", MessageType.mtTitulo)
    Display.ShowMessage("", MessageType.mtTitulo)
    AnaliseRespostasIA.AnalisarRespostas(self.respostasAnalise)
    ResultadosAnalise.Exibir(self, resultadosAnalise, "análise por Rede Neural")

In [59]:
# @title Executa a aplicação de questionário sobre conhecimentos fotográficos
#
AvaliacaoFotografo().IniciarAvaliacao()

#TO DO: Gerar respostas em arquivo texto
#TO DO: Criar um método

Com o intuito de avaliar suas capacidades fotográficas e o que conseguiu apreender durante
o curso básico de fotografia, além de guiá-lo no desenvolvimento de suas capacidades e ajudá-lo
a identificar seus pontos fortes e fracos, apresentamos a avaliação abaixo.

Leia atentamente as perguntas e as respostas e indique a que melhor responde às questões propostas.
Em tempo, não pense que todas as questões são feitas de certo/errado, pois em alguns casos podem 
existir várias respostas certas, de acordo com o seu nível de conhecimento. Nesse caso, escolha a que
achar a mais completa ou mais correta.

Vamos começar.



	1 - Deve sempre abranger todo o rosto da pessoa
	2 - Deve estar sempre cravado nos olhos da pessoa
	3 - Deve estar sempre na região mais próxima da câmera, evidenciando a profundidade de campo
	4 - Pode estar em qualquer lugar da foto
Digite sua resposta (ou 'S' para sair): 1


	1 - Quanto mais aberto o diafragma, maior a profundidade de campo
	2 - Quanto maior a velocidade do obturador, maior a profundidade de campo
	3 - Quanto mais fechado o diafragma, maior a profundidade de campo
	4 - Quanto maior o ISO, maior a profundidade de campo
Digite sua resposta (ou 'S' para sair): 1


	1 - Determina a sensibilidade do sensor à luz que chega até ele
	2 - Determina a quantidade de luz que chega ao sensor
	3 - Determina a velocidade com que o obturador abre e fecha
	4 - Determina se a fotografia ficará ou não no foco
Digite sua resposta (ou 'S' para sair): 1


	1 - Permite que o sensor perceba mais ou menos luz
	2 - Aumenta a profundidade de campo da cena
	3 - Possui como unidade de medida a letra f e quanto maior o valor, maior a quantidade de luz que chega ao sensor
	4 - É representada em segundos e suas frações e tem a capacidade de congelar a cena quando alta o suficiente
Digite sua resposta (ou 'S' para sair): 1


...

	1 - O assunto da foto deve sempre estar centralizado e o formato da foto deve ser quadrado
	2 - Pelo menos dois lados da foto devem ser simétricos, não importando a orientação e nem o formato da foto
	3 - Os resultados de fotografias simétricas não são muito agradáveis aos olhos do observador
	4 - Nunca pode ser utilizado em fotos de retrato porque o rosto humano nunca é simétrico
Digite sua resposta (ou 'S' para sair): 1


Você já tem um conhecimento básico e pode tirar boas fotos, mas ainda conta muito 
com o fator sorte para conseguir fotos com alguma qualidade.



Você consegue fazer boas fotos, mas mais estudo pode garantir a sua independência da sorte.




Você já tem um conhecimento básico e pode tirar boas fotos, mas ainda conta muito 
com o fator sorte para conseguir fotos com alguma qualidade.



Você consegue fazer boas fotos, mas mais estudo pode garantir a sua independência da sorte.




In [60]:
# @title Testes de criação, aprendizado e utilização da rede neural

class TesteRedeNeural:
  @staticmethod
  def Testar():
    knowledgeBase = KnowledgeBase().Add([1, 2, 3, 3, 3, 1], []) \
                                  .Add([2, 2, 2, 0], []) \
                                  .Finalize()
    print(knowledgeBase.Entradas)
    #raise Exception("Interrupção do código")

    analiseRespostasCategoria = ResultadosAnaliseBuilder.Build()
    listaPerguntas = PerguntasBuilder.Build()

    redeNeuralBuilder = RedeNeuralBuilder(listaPerguntas)
    #--------------------------------------------------------------------------
    loss = 0
    accuracy = 1
    evaluateResults = []
    lossAnterior = 1
    lossAtual = 0

    while lossAtual < lossAnterior:
      model = redeNeuralBuilder.SaveData() \
                              .UseSavedData() \
                              .ForceTrainingModel() \
                              .UseEpochs(1500) \
                              .WithLossMethod('binary_crossentropy') \
                              .Create()

      if lossAtual != 0:
        lossAnterior = lossAtual

      lossAtual = redeNeuralBuilder.loss
      evaluateResults.append([redeNeuralBuilder.loss,
                              redeNeuralBuilder.accuracy])

    print()
    for evaluateResult in evaluateResults:
      print(f"loss: {evaluateResult[loss]}")
      print(f"accuracy: {evaluateResult[accuracy]}")
    print()
    print()

    predictions = model.predict(knowledgeBase.Entradas)

    print(predictions)

    for i in range(len(knowledgeBase.Entradas)):
      resposta = round(float(predictions[i]), 2)*100
      categoria = next((categoria for categoria in listaPerguntas if categoria.Index == i + 1))
      analise = next((analise for analise in analiseRespostasCategoria if (resposta >= analise.Minimo and resposta <= analise.Maximo) and (analise.Categoria == i + 1)))

      print(f"As análises para a categoria {categoria.Caption} foi: {resposta}%")
      print()
      print(categoria.Caption)
      print(analise.Resposta)
      print()

#TesteRedeNeural.Testar()